# 05: Policy Simulation & Action Planning

This notebook demonstrates the policy intervention simulator and action recommender.

**Goal**: Answer "What if we deploy intervention X with budget B?"

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from simulator import (
    load_interventions, load_priority_data,
    simulate_intervention, run_monte_carlo_simulation
)
from action_recommender import recommend_actions, run_recommendation_engine

## 1. Load Intervention Definitions

In [ ]:
interventions = load_interventions('../config/interventions.json')

# Display interventions
for name, config in interventions.items():
    print(f"\n{name.upper()}")
    print(f"  Description: {config['description']}")
    print(f"  Cost: ₹{config['cost']:,}")
    print(f"  Capacity: {config['capacity_per_week']}/week")
    print(f"  Duration: {config['duration_weeks']} weeks")

## 2. Single District Simulation

In [ ]:
# Example district data
district_data = {
    'state': 'Andhra Pradesh',
    'district': 'Example District',
    'backlog': 2000,  # Current backlog
    'weekly_demand': 300,  # Expected weekly demand
    'baseline_capacity': 200  # Current capacity
}

# Simulate mobile camp intervention
result = simulate_intervention(
    district_data,
    interventions['mobile_camp'],
    'mobile_camp',
    scenario='median'
)

print(f"\nSimulation Result for {result.district}:")
print(f"  Intervention: {result.intervention}")
print(f"  Initial backlog: {result.initial_backlog:,.0f}")
print(f"  Final backlog: {result.final_backlog:,.0f}")
print(f"  Reduction: {result.reduction_pct:.1f}%")
print(f"  Cost per update: ₹{result.cost_per_update:.2f}")
print(f"  Fairness index: {result.fairness_index:.3f}")

## 3. Monte Carlo Simulation (Uncertainty)

In [ ]:
# Run Monte Carlo with uncertainty
mc_result = run_monte_carlo_simulation(
    district_data,
    interventions['mobile_camp'],
    'mobile_camp',
    n_runs=1000,
    forecast_lower=250,
    forecast_upper=400
)

print(f"\nMonte Carlo Results ({mc_result['n_runs']} runs):")
print(f"  Backlog reduction (median): {mc_result['backlog_reduction']['p50']:,.0f}")
print(f"  90% Confidence interval: [{mc_result['backlog_reduction']['p5']:,.0f}, {mc_result['backlog_reduction']['p95']:,.0f}]")
print(f"  Cost per update (median): ₹{mc_result['cost_per_update']['p50']:.2f}")

## 4. Compare All Interventions

In [ ]:
# Compare interventions for the same district
comparison = []

for name, config in interventions.items():
    result = simulate_intervention(district_data, config, name, 'median')
    comparison.append({
        'intervention': name,
        'cost': result.total_cost,
        'reduction_pct': result.reduction_pct,
        'cost_per_update': result.cost_per_update,
        'fairness_index': result.fairness_index
    })

comp_df = pd.DataFrame(comparison)
comp_df = comp_df.sort_values('cost_per_update')

# Visualize
fig = px.bar(
    comp_df, x='intervention', y='reduction_pct',
    color='cost_per_update', color_continuous_scale='RdYlGn_r',
    title='Intervention Comparison: Reduction % vs Cost Efficiency'
)
fig.show()

## 5. Budget-Constrained Optimization

In [ ]:
# Run recommendation engine with different budgets
budgets = [500000, 1000000, 2000000]

for budget in budgets:
    print(f"\n{'='*50}")
    print(f"Budget: ₹{budget:,}")
    print(f"{'='*50}")
    
    priority = load_priority_data('../outputs/priority_scores.csv', '../outputs/forecasts.csv')
    selected, summary = recommend_actions(priority.head(30), interventions, budget)
    
    print(f"  Actions selected: {summary['actions_selected']}")
    print(f"  Districts covered: {summary['districts_covered']}")
    print(f"  Budget used: ₹{summary['budget_used']:,.0f}")
    print(f"  Est. total reduction: {summary['total_backlog_reduction']:,.0f}")

## 6. Visualize Action Plan

In [ ]:
# Load recommended actions
try:
    with open('../outputs/sim_results/recommended_actions.json') as f:
        recommendations = json.load(f)
    
    actions_df = pd.DataFrame(recommendations['selected_actions'])
    
    # Plot
    fig = px.scatter(
        actions_df, 
        x='cost', y='backlog_reduction',
        color='intervention', size='priority_score',
        hover_data=['district', 'state'],
        title='Recommended Actions: Cost vs Impact'
    )
    fig.update_layout(xaxis_title='Cost (₹)', yaxis_title='Backlog Reduction')
    fig.show()
    
except FileNotFoundError:
    print("Run action_recommender.py first to generate recommendations")

## 7. Sample Action Pack

In [ ]:
# Load and display a sample action pack
import glob

action_packs = glob.glob('../outputs/sim_results/action_pack_*.json')

if action_packs:
    with open(action_packs[0]) as f:
        pack = json.load(f)
    
    print("SAMPLE ACTION PACK")
    print("="*50)
    print(f"District: {pack['meta']['district']} ({pack['meta']['state']})")
    print(f"Intervention: {pack['intervention_details']['name']}")
    print(f"\nRationale:")
    print(f"  Bottleneck: {pack['rationale']['bottleneck_type']}")
    print(f"  Priority Score: {pack['rationale']['priority_score']:.2f}")
    print(f"\nExpected Impact:")
    print(f"  Reduction: {pack['expected_impact']['backlog_reduction']['median']:.0f}")
    print(f"  90% CI: {pack['expected_impact']['backlog_reduction']['range_90pct']}")
    print(f"\nCost: ₹{pack['cost_breakdown']['total_cost']:,}")
else:
    print("No action packs found. Run action_recommender.py first.")